In [5]:
import pandas as pd
import datetime as dt
import requests 
import math
from datetime import datetime, timedelta
import pytz
from pytz import timezone
import calendar

In [615]:
TOTAL = pd.read_stata('individual_new.dta')  #ORIGINAL DATA IMPORT TO PYTHON LIKE A PANDAS DATAFRAME
TOTAL.head(5)

C:\Users\davalvar\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\stata.py:1514: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  warnings.warn(msg, UnicodeWarning)


,country,isocode,ison,region,language,date,id_gallup,wgt,patience,risktaking,posrecip,negrecip,altruism,trust,subj_math_skills,gender,age
0,Turkey,TUR,792.0,Adana,Turkish,2012-03-07,7.100800e+12,0.271783,0.047176,1.020203,0.594384,-0.367175,-0.139953,1.679754,7.0,1,26
1,Turkey,TUR,792.0,Adana,Turkish,2012-03-08,7.100800e+12,0.271783,-0.675698,0.387177,0.662234,0.077251,-0.139953,0.950434,3.0,1,50
2,Turkey,TUR,792.0,Adana,Turkish,2012-03-08,7.100800e+12,0.442259,0.318254,1.020203,-0.000930,0.077251,-0.606967,0.585774,7.0,1,21
3,Turkey,TUR,792.0,Adana,Turkish,2012-03-07,7.100800e+12,1.423671,0.498972,1.271528,0.959891,0.077251,0.560569,0.585774,7.0,0,24
4,Turkey,TUR,792.0,Adana,Turkish,2012-03-07,7.100800e+12,0.705356,0.589331,1.122619,1.325398,-0.367175,0.327062,1.679754,9.0,0,24


In [616]:
DST_2012= pd.read_csv('DST_2012.txt')  #DAYLIGHT SAVING TIME (DST), FOR ALL COUNTRIES IN THE YEAR 2012
DST_2012 = DST_2012.reset_index()
DST_2012.head()

,index,Country,DST Start Date,DST End Date,Unnamed: 3
0,0,Afghanistan,Not observed,Not observed,NaN
1,1,Albania,2012-03-25,2012-10-28,NaN
2,2,Algeria,Not observed,Not observed,NaN
3,3,Andorra,2012-03-25,2012-10-28,NaN
4,4,Angola,Not observed,Not observed,NaN


In [617]:
UTC_df = pd.read_csv('UTC.txt')  # TIME ZONES IN WHICH THE PLANET IS DIVIDED
UTC_df.head()

,UTC,SINCE,TO,Unnamed: 3
0,UTC-12,-180.0,-172.5,NaN
1,UTC-11,-172.5,-157.5,NaN
2,UTC-10,-157.5,-142.5,NaN
3,UTC-9,-142.5,-127.5,NaN
4,UTC-8,-127.5,-112.5,NaN


In [620]:
BASE_URL = "http://api.weatherapi.com/v1/history.json?" # URL OF THE API USED TO OBTAIN THE DATA
API_KEY =  API_KEY  # API KEY
url = BASE_URL+ "key=" + API_KEY +"&q=" + CITY +"&dt=2012-03-26" 
response= requests.get(url).json()
response                               #VIEW THE DATA OBTAINED FROM THE USED API

{'location': {'name': 'Paris',
  'region': 'Ile-de-France',
  'country': 'France',
  'lat': 48.87,
  'lon': 2.33,
  'tz_id': 'Europe/Paris',
  'localtime_epoch': 1678291264,
  'localtime': '2023-03-08 17:01'},
 'forecast': {'forecastday': [{'date': '2012-03-26',
    'date_epoch': 1332720000,
    'day': {'maxtemp_c': 19.7,
     'maxtemp_f': 67.5,
     'mintemp_c': 8.0,
     'mintemp_f': 46.4,
     'avgtemp_c': 15.0,
     'avgtemp_f': 59.0,
     'maxwind_mph': 12.1,
     'maxwind_kph': 19.4,
     'totalprecip_mm': 0.0,
     'totalprecip_in': 0.0,
     'avgvis_km': 10.0,
     'avgvis_miles': 6.0,
     'avghumidity': 64.0,
     'condition': {'text': 'Sunny',
      'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png',
      'code': 1000},
     'uv': 5.0},
    'astro': {'sunrise': '06:40 AM',
     'sunset': '07:13 PM',
     'moonrise': '07:59 AM',
     'moonset': '11:34 PM',
     'moon_phase': 'Waxing Crescent',
     'moon_illumination': '10'},
    'hour': [{'time': '2012-03-26 00:00', '

In [ ]:
def DST_inter(pais):     #  FUNCTION USED TO KNOW IF A DATE IS WITHIN THE DST TIME OR NOT,
                         #IF IT IS WITHIN THE DST TIME IT RETURNS START DATE AND END DATE OF THE COUNTRY'S DST TIME.

    dst_i = DST_2012.loc[DST_2012['Country'] == pais, 'DST Start Date' ].values[0]
    dst_f = DST_2012.loc[DST_2012['Country'] == pais, 'DST End Date' ].values[0]
    
    if dst_i == 'Not observed':
        return (' ',' ')
    else:
        dst_i = datetime.datetime.strptime(dst_i, '%Y-%m-%d')  
        dst_f = datetime.datetime.strptime(dst_f, '%Y-%m-%d')  
       
    
    return (dst_i,dst_f)



In [ ]:
                                    #FUNCTION USED TO CALCULATE THE AVERAGE SUNRISE AND SUNSET TIMES
                                    #DURING A YEAR FOR A GIVEN COUNTRY, 
                                    #TIME ADJUSTED BY DST TIME

def avg_sunrise_sunset_t(COUNTRY):
    sunrise_day = []
    sunset_day = [] 

    for au in range(1,13):
        dia = '30'
        if au <= 9:
            mes = '0'+str(au)  

            if au == 2:
                dia = '28'          
        else:
            mes = str(au)  

            
        url = BASE_URL+ "key=" + API_KEY +"&q=" + COUNTRY +"&dt=2012-"+ mes +"-01" +"&end_dt=2012-"+ mes + '-' + dia
        response= requests.get(url).json()

        for i in range(0,int(dia)-1):
            date1 = response['forecast']['forecastday'][i]['date']  
            time1_rise = response['forecast']['forecastday'][i]['astro']['sunrise']
            time1_set = response['forecast']['forecastday'][i]['astro']['sunset']
            
            time2_rise = datetime.datetime.strptime(date1 + ' ' + time1_rise, '%Y-%m-%d %H:%M AM')
            time2_set = datetime.datetime.strptime(date1 + ' ' + time1_set, '%Y-%m-%d %H:%M PM')+timedelta(hours = 12)   
            
            sunrise_day.append(time2_rise)
            sunset_day.append(time2_set)

    df_sunrise_day = pd.DataFrame(sunrise_day)
    df_sunrise_day.columns =['sunrise_all']  
    
    df_sunset_day = pd.DataFrame(sunset_day)
    df_sunset_day.columns =['sunset_all']  
    
    A, B = DST_inter(COUNTRY)  # IT IS NECESSARY TO DETERMINE WHETHER OR NOT THE SUNRISES AND SUNSETS WERE IN DST TIME

    if A != ' ' and B != ' ':
        df_sunrise_day.loc[df_sunrise_day['sunrise_all'].between(A, B), 'sunrise_all'] = df_sunrise_day['sunrise_all']+timedelta(hours = 1)
        avg_sunrise = df_sunrise_day['sunrise_all'].mean()
        
        df_sunset_day.loc[df_sunset_day['sunset_all'].between(A, B), 'sunset_all'] = df_sunset_day['sunset_all']+timedelta(hours = 1)   
        avg_sunset = df_sunset_day['sunset_all'].mean()
        
        return(avg_sunrise,avg_sunset)
    else:
        avg_sunrise = df_sunrise_day['sunrise_all'].mean()
        avg_sunset = df_sunset_day['sunset_all'].mean()
        
        return(avg_sunrise,avg_sunset)

In [ ]:
def wind_pro():    # AVERAGE WEATHER SPEED ON A GIVEN DAY 
    win_aux = 0
    for i in range(23):
        win_aux += float(response['forecast']['forecastday'][0]['hour'][i]['wind_kph'])
        win = win_aux/24  
    return round(win,2) 

In [ ]:
def hours_sun():  # CALCULATION OF THE HOURS OF SUNSHINE DURING A GIVEN DAY, IN A GIVEN COUNTRY AND REGION
    sunrise =  response['forecast']['forecastday'][0]['astro']['sunrise']
    sunset_m = response['forecast']['forecastday'][0]['astro']['sunset'][3:5]
    sunset_h = int(response['forecast']['forecastday'][0]['astro']['sunset'][0:2])+12
    sunset = str(sunset_h) +':'+str(sunset_m)
    

    sunrise_obj = datetime.datetime.strptime(sunrise, '%H:%M AM')
    sunset_obj = datetime.datetime.strptime(sunset, '%H:%M')
    aux = sunset_obj - sunrise_obj
    hour1 = str(aux)
    if hour1[1:2] == ':':
        hour2 = str(23 - int(hour1[0:1])) + ':' + str(60 - int(hour1[3:4])) 
    else:
        hour2 = str(23 - int(hour1[0:2])) + ':' + str(60 - int(hour1[3:5]))  
    return hour1, hour2

In [ ]:
def DST(pais,date):      #FUNCTION ONLY TO CALCULATE WHETHER OR NOT THE DATE 
                         #ON WHICH THE DATA WAS TAKEN IS WITHIN OR OUTSIDE THE DST SCHEDULE
       
    dst_i = DST_2012.loc[DST_2012['Country'] == pais, 'DST Start Date' ].values[0]
    dst_f = DST_2012.loc[DST_2012['Country'] == pais, 'DST End Date' ].values[0]
    
    if dst_i == 'Not observed':
        return ' '
    
    dst_i = datetime.datetime.strptime(dst_i, '%Y-%m-%d')  
    dst_f = datetime.datetime.strptime(dst_f, '%Y-%m-%d')  
    date = datetime.datetime.strptime(date, '%Y-%m-%d')
    
    if dst_i < date < dst_f :
        return 'DST'
    else:
        return ' ' 

In [ ]:
def UTC(lat, long):    # FUNCTION I CALCULATE WHICH WOULD BE THE CLOSEST TIME ZONE TO THE CITY EVALUATED AND WHETHER IT IS TO THE EAST OR WEST.
    
    
    
    zone = response['location']['tz_id']
    now = datetime.datetime.now(pytz.timezone(zone))
    #return now.strftime('%z')

    
    level = UTC_df.loc[(( UTC_df['SINCE'] <= long ) & ( UTC_df['TO'] >= long)), 'UTC' ].values[0] 
    
    left = long - UTC_df.loc[UTC_df['UTC'] == level  , 'SINCE' ].values[0]
    right = UTC_df.loc[UTC_df['UTC'] == level  , 'TO' ].values[0] - long 
    
    if left >= right:
        return('UTC' + now.strftime('%z'), round(right*111.3, 2))
    else:
        return('UTC' + now.strftime('%z'), -1*round(left*111.3, 2)) 
    

In [ ]:
def sun_angle(latitude, longitude, date_str):
    time_str = '12:00:00'
    # Convert the date and time string to a datetime object
    tz = timezone('UTC') # Use UTC timezone
    dt = datetime.datetime.strptime(date_str + ' ' + time_str, '%Y-%m-%d %H:%M:%S')
    dt_utc = tz.localize(dt) # Make the datetime object timezone-aware

    # Calculate the Julian day (JD) for the given date and time
    year = dt_utc.year
    month = dt_utc.month
    day = dt_utc.day
    hour = dt_utc.hour
    minute = dt_utc.minute
    second = dt_utc.second
    jd = (367 * year) - int((7 * (year + int((month + 9) / 12))) / 4) + int((275 * month) / 9) + day + 1721013.5 + ((hour + (minute / 60) + (second / 3600)) / 24)

    # Calculate the Julian century (JC) from the JD
    jc = (jd - 2451545) / 36525

    # Calculate the solar mean anomaly (M) for the JC
    m = (357.5291 + (35999.0503 * jc)) % 360

    # Calculate the equation of the center (C) for the JC
    c = (1.9148 * math.sin(math.radians(m))) + (0.02 * math.sin(math.radians(2 * m))) + (0.0003 * math.sin(math.radians(3 * m)))

    # Calculate the ecliptic longitude (lambda) for the JC
    lmbda = (m + 102.9372 + c + 180) % 360

    # Calculate the solar declination (delta) for the JC
    delta = math.asin(math.sin(math.radians(23.45)) * math.sin(math.radians(lmbda)))

    # Calculate the local hour angle (H) for the given longitude and time
    ha = math.acos((math.sin(math.radians(-0.83)) - (math.sin(math.radians(latitude)) * math.sin(delta))) / (math.cos(math.radians(latitude)) * math.cos(delta)))

    # Calculate the solar altitude angle (theta) for the given latitude, declination, and hour angle
    theta = math.degrees(math.asin((math.cos(math.radians(ha)) * math.cos(math.radians(delta)) * math.cos(math.radians(latitude))) + (math.sin(math.radians(delta)) * math.sin(math.radians(latitude)))))

    return round(theta, 2)

In [ ]:
df_ordenado = TOTAL

df_country_unico = df_ordenado.groupby('country').sum().reset_index()     # THE LIST IS SORTED BY COUNTRY TO WORK WITH ONE COUNTRY AT A TIME
df_country_unico = df_country_unico.sort_values('country', ascending=True)   

for  index, row in df_country_unico.iterrows():
    countryf = df_country_unico['country'].values[index] 
    print(countryf)
    
    df_country_unico_a = df_ordenado[df_ordenado['country'] == df_country_unico['country'].values[index]] # THE LIST OF COUNTRIES IS SORTED BY REGION TO WORK ONE REGION AT A TIME
    df_region_unica = df_country_unico_a.groupby('region').sum().reset_index()
    df_region_unica = df_region_unica.sort_values('region', ascending=True)

    for index, row in df_region_unica.iterrows():

        regionf = df_region_unica['region'].values[index] 
        ciudad = countryf + ',' + regionf 

        print(ciudad)
 
        df_region_a = df_ordenado[df_ordenado['region']== df_region_unica['region'].values[index]] # THE LIST OF REGIONS SORTED BY DAY TO WORK ONE DAY AT A TIME
        df_tiempo = df_region_a.groupby('date').sum().reset_index()
        df_tiempo.sort_values('date', ascending=False)


        for index, row in  df_tiempo.iterrows():
            print(index)
            datef = df_tiempo['date'].values[index]
            day = datetime.datetime.strptime(str(datef)[:10], '%Y-%m-%d').weekday() # FUNCTION TO KNOW WHICH DAY OF THE WEEK WAS THE DAY OF THE WEEK 


            
            url = BASE_URL+ "key=" + API_KEY +"&q=" + ciudad +"&dt=" + str(datef)[:10]    #THE DATA IS REQUESTED BY COUNTRY, REGION AND DAY 
            response= requests.get(url).json()                                            
                                              
            
            if response['location']['country'] != countryf:                               #AND THE SELECTED COUNTRY IS VERIFIED TO BE CORRECT.
                
                letter = '('
                ptts = ciudad.rfind(letter)
                
                if ptts != -1:
                     ciudad = ciudad[:ptts]
                url = BASE_URL+ "key=" + API_KEY +"&q=" + ciudad +"&dt=" + str(datef)[:10]        
                response= requests.get(url).json()
                
                if response['location']['country'] != countryf:
                    url = BASE_URL+ "key=" + API_KEY +"&q=" + countryf +"&dt=" + str(datef)[:10]
                    response= requests.get(url).json()
    

 
            lat = response['location']['lat']         #ASSIGNMENT OF ALL VALUES IN THE REQUESTED ORDER
            lon = response['location']['lon']
  
            df_ordenado.loc[(( df_ordenado['country'] ==  countryf) & ( df_ordenado['region'] == regionf)),
                ['latitude','longitude'] ] = (lat, lon) 
            
            tempe_day = response['forecast']['forecastday'][0]['day']['avgtemp_c']                        
            pre_mm = response['forecast']['forecastday'][0]['day']['totalprecip_mm']        
            clouds = response['forecast']['forecastday'][0]['day']['condition']['text']
            Sunrise  =  response['forecast']['forecastday'][0]['astro']['sunrise']
            sunset  =  response['forecast']['forecastday'][0]['astro']['sunset']
            Moonrise = response['forecast']['forecastday'][0]['astro']['moonrise']
            Moonset  = response['forecast']['forecastday'][0]['astro']['moonset']
            DST_f =  DST(countryf,str(datef)[:10])     
            if DST_f  == 'DST':
                if Sunrise != 'No sunrise':   # #THE MOONRISE OR MOONSET SOMETHIMES IS NOT VISIBLE
                    Sunrise_s  = datetime.datetime.strptime(Sunrise, '%H:%M ' + str(Sunrise[6:8]))+timedelta(hours = 1)
                    Sunrise = str(Sunrise_s.time())+' ' + str(Sunrise[6:8])
                if sunset != 'No sunset':                     
                    sunset_s  =  datetime.datetime.strptime(sunset, '%H:%M ' + str(sunset[6:8]))+timedelta(hours = 1)
                    sunset = str(sunset_s.time())+ ' '+ str(sunset[6:8])
                if Moonrise != 'No moonrise':  
                    Moonrise_s =  datetime.datetime.strptime(Moonrise, '%H:%M ' + str(Moonrise[6:8]) )+timedelta(hours = 1)
                    Moonrise =  str(Moonrise_s.time()) + ' ' + str(Moonrise[6:8])
                if Moonset != 'No moonset':  
                    Moonset_s  = datetime.datetime.strptime(Moonset, '%H:%M ' + str(Moonset[6:8]))+timedelta(hours = 1)
                    Moonset =  str(Moonset_s.time()) + ' ' + str(Moonset[6:8])

     
                
            Moon_phase = response['forecast']['forecastday'][0]['astro']['moon_phase']
            date_g =   response['forecast']['forecastday'][0]['date']
            hour1, hour2 =  hours_sun() 
            sun__angle  = sun_angle(lat, lon, str(datef)[:10]) 

            UTC_T, KM = UTC(lat, lon)     

            df_ordenado.loc[(( df_ordenado['country'] ==  countryf) & 
                             ( df_ordenado['region'] == regionf ) & 
                              (df_ordenado['date'] == datef)) ,
            ['Day of the week','Temperature', 'Wind Speed','Precipitation in mm', 'Hours of sunlight obtained', 'Clouds' ,'Sun Angle at 12:00' ,'Moonrise','Moonset' , 'Moon phase', 'Sunrise' ,'sunset','Dark hours' ,'AVG sunrise', 'AVG sunset' , 'DST']]=(calendar.day_name[day] , tempe_day, wind_pro(),  pre_mm,  hour1,  clouds , sun__angle ,Moonrise,Moonset, Moon_phase , Sunrise , sunset, hour2 , ' ' ,' ' , DST_f )
   

        avg_sunrise, avg_sunset =  avg_sunrise_sunset_t(countryf) 
        
        df_ordenado.loc[(( df_ordenado['country'] ==  countryf) & ( df_ordenado['region'] == regionf)),
                        ['DST','Near tz km', 'UTC'  ]] = (DST(countryf,str(datef)[:10]) ,KM, UTC_T)
        
        df_ordenado.loc[(( df_ordenado['country'] ==  countryf) & ( df_ordenado['region'] == regionf)),
                        ['AVG sunrise', 'AVG sunset']] = (str(avg_sunrise)[11:19], str(avg_sunset)[11:19])



In [16]:
information_final = df_ordenado    #FINAL INFORMATION READY FOR EXPORTATION     
information_final.to_stata('information_final.dta') 